# SDG 15.3.1 Error Recode Script Example

This notebook demonstrates how to use the SDG 15.3.1 error recode script via the trends.earth API. The script supports both single-period and multi-period land degradation jobs and allows for recoding specific areas based on error polygons.

## 1. Setup Environment and Import Libraries

Before running this notebook, make sure you have:

1. **Installed dependencies**: `pip install -r requirements.txt`
2. **Created environment file**: Copy `.env.example` to `.env` and update with your credentials:
   ```bash
   cp .env.example .env
   ```
3. **Updated credentials** in the `.env` file:
   ```
   API_BASE_URL=https://api.trends.earth
   API_USERNAME=your_actual_username
   API_PASSWORD=your_actual_password
   ```

In [ ]:
import json
import uuid
import warnings
from pprint import pprint

import matplotlib.pyplot as plt
import numpy as np
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Import the shared API client
import folium
import rasterio
import rasterio.mask
from trendsearth_api import TrendsEarthAPIClient

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

print("Libraries imported successfully!")
print("Environment variables loaded from .env file")

Libraries imported successfully!
Environment variables loaded from .env file


## 2. Configure API Connection and Authentication

In [2]:
# Initialize the API client
api_client = TrendsEarthAPIClient()

# Authenticate using environment variables from .env file
if api_client.authenticate_from_env():
    print("Ready to submit error recode jobs!")
    session = api_client.session  # For compatibility with existing code
else:
    print("Authentication failed - check your .env file")
    print("Please ensure your .env file contains:")
    print("- API_USERNAME=your_username")
    print("- API_PASSWORD=your_password")
    print("- API_BASE_URL=https://api.trends.earth (optional)")
    session = None

Using API URL from environment: https://api.trends.earth
Authenticating with Trends.Earth API...
   Using email: trends.earth-prais-server@trends.earth
Successfully authenticated with Trends.Earth API
Ready to submit error recode jobs!


## 3. Define Example Parameters for Error Recoding

### Multi-period Data Support

The  error recode script uses the `filters` parameter to select specific reporting periods for multi-period land degradation jobs:

**For Single-period Jobs:**
```python
"filters": []  # No filters needed
```

**For Multi-period Jobs:**
```python
"filters": [{"field": "reporting_year_final", "value": 2020}]  # Target specific year
```

### Error Polygon Schema Requirements

The error polygons must follow the trends.earth schema validation requirements:

**Required Properties:**
- `uuid`: A unique identifier (auto-generated UUID string)
- `type`: Must be "Feature" for each feature

**Optional Properties:**
- `location_name`: Descriptive name for the error area
- `area_km_sq`: Area in square kilometers
- `process_driving_change`: Description of what caused the error
- `basis_for_judgement`: Justification for the correction

**Recode Options** (all optional, use None to leave unchanged):
- `recode_deg_to`: What to recode degraded pixels to
  - `None`: No change (default)
  - `-32768`: No data
  - `0`: Stable
  - `1`: Improved
- `recode_stable_to`: What to recode stable pixels to
  - `None`: No change (default) 
  - `-32768`: No data
  - `-1`: Degraded
  - `1`: Improved
- `recode_imp_to`: What to recode improved pixels to
  - `None`: No change (default)
  - `-32768`: No data
  - `-1`: Degraded
  - `0`: Stable

In [3]:
with open("ATG_polygon.geojson", "r") as f:
    atg_polygon = json.load(f)

params = {
    "iso": "ATG",
    "boundary_dataset": "UN",
    "write_tifs": False,
    "aoi": json.dumps(atg_polygon),
    "error_polygons": {
        "type": "FeatureCollection",
        "name": "test error recode",
        "crs": {
            "type": "name",
            "properties": {"name": "urn:ogc:def:crs:OGC:1.3:CRS84"},
        },
        "features": [
            {
                "type": "Feature",
                "properties": {
                    "uuid": str(uuid.uuid4()),
                    "location_name": "False degradation - mining misclassified",
                    "area_km_sq": 0.04,
                    "process_driving_change": "Open pit mining misclassified as degradation",
                    "basis_for_judgement": "Ground truth verification shows managed mining operation",
                    "recode_deg_to": 0,  # Recode degraded pixels to stable
                    "recode_stable_to": None,  # No change to stable pixels
                    "recode_imp_to": None,  # No change to improved pixels
                    "periods_affected": ["baseline"],
                },
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [
                        [
                            [-61.85, 17.05],
                            [-61.8, 17.05],
                            [-61.8, 17.1],
                            [-61.85, 17.1],
                            [-61.85, 17.05],
                        ]
                    ],
                },
            },
            {
                "type": "Feature",
                "properties": {
                    "uuid": str(uuid.uuid4()),
                    "location_name": "Restoration success not captured",
                    "area_km_sq": 0.02,
                    "process_driving_change": "Successful restoration not detected by algorithm",
                    "basis_for_judgement": "Field monitoring shows vegetation recovery",
                    "recode_deg_to": 1,  # Recode degraded pixels to improved
                    "recode_stable_to": 1,  # Also recode stable pixels to improved
                    "recode_imp_to": None,  # Keep improved pixels as is
                    "periods_affected": ["baseline", "reporting_2"],
                },
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [
                        [
                            [-61.82, 17.08],
                            [-61.78, 17.08],
                            [-61.78, 17.12],
                            [-61.82, 17.12],
                            [-61.82, 17.08],
                        ]
                    ],
                },
            },
            {
                "type": "Feature",
                "properties": {
                    "uuid": str(uuid.uuid4()),
                    "location_name": "Data quality issue",
                    "area_km_sq": 0.01,
                    "process_driving_change": "Cloud contamination in satellite data",
                    "basis_for_judgement": "Visual inspection shows persistent cloud cover",
                    "recode_deg_to": -32768,  # Set all to no data due to poor data quality
                    "recode_stable_to": -32768,  # Set all to no data
                    "recode_imp_to": -32768,  # Set all to no data
                    "periods_affected": ["reporting_2"],
                },
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [
                        [
                            [-61.88, 17.15],
                            [-61.84, 17.15],
                            [-61.84, 17.18],
                            [-61.88, 17.18],
                            [-61.88, 17.15],
                        ]
                    ],
                },
            },
        ],
    },
}

## 4. Submit Job Execution to API

In [ ]:
if api_client.access_token:
    print(" Submitting error recode job...")
    execution_id = api_client.submit_job("sdg-15-3-1-error-recode-2-1-17", params)

    if execution_id:
        print("Job submitted. Monitoring job status...")

        # Monitor the job until completion
        final_status = api_client.monitor_job(execution_id, max_minutes=15)

        if final_status:
            status = final_status.get("status", "unknown").upper()
            if status in ["SUCCESS", "FINISHED"]:
                print("✅ Error recode job completed successfully!")

                # The results should be available in the job data
                results = final_status.get("results", {})
                if results:
                    print("Results available")
                    pprint(results)
                else:
                    print("No results data found in job response")
        else:
            print("❌ Failed to monitor job")
    else:
        print("❌ Failed to submit job")
else:
    print("❌ Cannot submit job: not authenticated")
    print("Please check your credentials and authentication")

Job submitted. Monitoring job status...
Status: READY
Status: FAILED
Job failed!


## 6. Display JSON Results

In [ ]:
def display_results_summary(results):
    """Display a summary of the error recode results"""

    if not results:
        print("No results to display")
        return

    print("=" * 60)
    print("ERROR RECODE RESULTS SUMMARY")
    print("=" * 60)

    # General information
    print("\n📊 General Information:")
    print(f"   Status: {results.get('status', 'Unknown')}")

    # Handle nested data structure
    data_section = results.get("data", results)

    # Processing information
    print("\n📝 Processing Information:")
    if "report" in data_section:
        print(f"   Report: {data_section['report']}")

    if "input_job" in data_section:
        input_job = data_section["input_job"]
        if isinstance(input_job, dict):
            print(f"   Task Name: {input_job.get('task_name', 'Unknown')}")
            print(
                f"   Input Band Index: {data_section.get('input_band_index', 'Unknown')}"
            )

    # Look for results in both possible locations
    raster_info = results.get("results") or data_section.get("results")

    if raster_info:
        print("\n🗺️ Raster Output:")
        print(f"   URI: {raster_info.get('uri', 'No URI available')}")

        if "bands" in raster_info:
            bands = raster_info["bands"]
            print(f"   Bands ({len(bands)}):")
            for i, band in enumerate(bands, 1):
                print(f"     {i}. {band.get('name', 'Unnamed band')}")
                print(f"        Activated: {band.get('activated', False)}")
                if "metadata" in band:
                    metadata = band["metadata"]
                    if "reporting_year_final" in metadata:
                        print(
                            f"        Reporting Year: {metadata['reporting_year_final']}"
                        )

    # Execution details (from API response)
    if "id" in results:
        print("\n🔧 Execution Details:")
        print(f"   Execution ID: {results.get('id')}")
        print(f"   Script ID: {results.get('script_id', 'Unknown')}")
        print(f"   User ID: {results.get('user_id', 'Unknown')}")

        if "created_at" in results:
            print(f"   Created: {results['created_at']}")
        if "start_time" in results:
            print(f"   Started: {results['start_time']}")
        if "end_time" in results:
            print(f"   Finished: {results['end_time']}")

    print("\n" + "=" * 60)


# Display the results
if "results" in locals() and results:
    display_results_summary(results)

    print("\n📋 Full JSON Results:")
    result_str = json.dumps(results, indent=2)
    if len(result_str) > 1000:
        print(result_str[:1000] + "...")
        print(
            f"\n[Truncated - showing first 1000 characters of {len(result_str)} total]"
        )
    else:
        print(result_str)
else:
    print("⚠️ No results available to display")

## 7. Load and Visualize Raster Data

In [ ]:
def create_error_recode_mask(error_polygons, raster_shape, extent):
    """Create a mask showing error recode areas"""

    mask = np.zeros(raster_shape)

    # Extract coordinates for demonstration
    x_min, y_max, x_max, y_min = extent

    # Handle GeoJSON FeatureCollection structure
    features = error_polygons.get("features", [])

    for i, feature in enumerate(features):
        geometry = feature["geometry"]
        properties = feature["properties"]
        coords = geometry["coordinates"][0]

        # Use property ID or index for recode value
        recode_value = properties.get("id", i + 1)

        # Simple rectangular mask for demonstration
        # In real implementation, you'd use proper polygon rasterization
        for coord in coords:
            lon, lat = coord

            # Convert to array indices
            col = int((lon - x_min) / (x_max - x_min) * raster_shape[1])
            row = int((y_max - lat) / (y_max - y_min) * raster_shape[0])

            # Create a small rectangular area around each coordinate
            for dr in range(-5, 6):
                for dc in range(-5, 6):
                    r, c = row + dr, col + dc
                    if 0 <= r < raster_shape[0] and 0 <= c < raster_shape[1]:
                        mask[r, c] = recode_value + 2  # Offset for visualization

    return mask


def load_raster_from_uri(uri):
    """Load raster data from S3 URI or local path"""
    try:
        # For demonstration, we'll create synthetic data
        # In real usage, you would load from the actual URI
        print(f"📁 Loading raster from: {uri}")

        if uri.startswith("s3://"):
            print("⚠️ S3 loading requires additional setup (AWS credentials, etc.)")
            print("Creating synthetic data for demonstration...")

            # Create synthetic land degradation data for Antigua and Barbuda coordinates
            rows, cols = 100, 100
            x = np.linspace(-61.9, -61.7, cols)  # Longitude range for ATG
            y = np.linspace(17.2, 17.0, rows)  # Latitude range for ATG

            # Create synthetic degradation pattern
            X, Y = np.meshgrid(x, y)

            # Base degradation pattern
            base_data = np.sin((X + 61.8) * 100) * np.cos((Y - 17.1) * 100) * 2

            # Add some areas of improvement and degradation
            degraded_mask = (X > -61.8) & (Y < 17.1)
            improved_mask = (X < -61.85) & (Y > 17.15)

            raster_data = np.where(
                degraded_mask,
                -1,  # Degraded
                np.where(
                    improved_mask,
                    1,  # Improved
                    0,
                ),
            )  # Stable

            # Add some noise
            noise = np.random.normal(0, 0.1, raster_data.shape)
            raster_data = raster_data + noise

            # Clip to valid range
            raster_data = np.clip(raster_data, -1, 1)

            return raster_data, (
                x.min(),
                y.max(),
                x.max(),
                y.min(),
            )  # extent for plotting

        else:
            # Load from local file
            if rasterio:
                with rasterio.open(uri) as src:
                    data = src.read(1)  # Read first band
                    extent = [
                        src.bounds.left,
                        src.bounds.right,
                        src.bounds.bottom,
                        src.bounds.top,
                    ]
                    return data, extent
            else:
                print("⚠️ Rasterio not available for local file loading")
                return None, None

    except Exception as e:
        print(f"❌ Error loading raster: {e}")
        return None, None


# Load the raster data
if "results" in locals() and results:
    # Handle both possible response structures
    raster_data_section = results.get("results") or results.get("data", {}).get(
        "results"
    )
    if raster_data_section and "uri" in raster_data_section:
        raster_uri = raster_data_section["uri"]
    else:
        raster_uri = "synthetic_data"  # Use synthetic data for demonstration
else:
    raster_uri = "synthetic_data"  # Use synthetic data for demonstration

print("\n🗺️ Loading raster data...")
raster_data, extent = load_raster_from_uri(raster_uri)

if raster_data is not None:
    print("✅ Raster loaded successfully!")
    print(f"   Shape: {raster_data.shape}")
    print(f"   Data range: {raster_data.min():.3f} to {raster_data.max():.3f}")
    print(f"   Extent: {extent}")

    # Create error recode mask using the corrected error polygons structure
    error_polygons_data = params["error_polygons"]
    error_mask = create_error_recode_mask(
        error_polygons_data, raster_data.shape, extent
    )
    print(f"   Error mask created with {np.sum(error_mask > 0)} pixels to recode")

else:
    print("❌ Failed to load raster data")

## 8. Create Simple Maps from Raster Results

Now let's create several visualization maps to show:
1. Original land degradation status
2. Error polygon locations and recode values  
3. Side-by-side comparison
4. Interactive map (optional)

In [ ]:
# Create comprehensive visualization maps
if raster_data is not None:
    # Set up the figure with subplots
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle(
        "SDG 15.3.1 Error Recode Analysis Results", fontsize=16, fontweight="bold"
    )

    # Define color maps and value ranges
    degradation_colors = ["darkred", "red", "yellow", "lightgreen", "darkgreen"]
    degradation_cmap = mcolors.ListedColormap(degradation_colors)
    degradation_bounds = [-1, -0.5, -0.1, 0.1, 0.5, 1]
    degradation_norm = mcolors.BoundaryNorm(degradation_bounds, degradation_cmap.N)

    # 1. Original Land Degradation Status
    ax1 = axes[0, 0]
    im1 = ax1.imshow(
        raster_data,
        extent=extent,
        cmap=degradation_cmap,
        norm=degradation_norm,
        aspect="auto",
    )
    ax1.set_title("Original Land Degradation Status", fontweight="bold")
    ax1.set_xlabel("Longitude")
    ax1.set_ylabel("Latitude")

    # Add colorbar
    cbar1 = plt.colorbar(im1, ax=ax1, shrink=0.8)
    cbar1.set_label("Degradation Index")
    cbar1.set_ticks([-0.75, -0.3, 0, 0.3, 0.75])
    cbar1.set_ticklabels(["Degraded", "Declining", "Stable", "Improving", "Improved"])

    # 2. Error Polygon Locations
    ax2 = axes[0, 1]
    # Show base raster in grayscale
    ax2.imshow(raster_data, extent=extent, cmap="gray", alpha=0.3, aspect="auto")

    # Overlay error mask
    error_cmap = plt.cm.Reds
    masked_errors = np.ma.masked_where(error_mask == 0, error_mask)
    im2 = ax2.imshow(
        masked_errors, extent=extent, cmap=error_cmap, alpha=0.8, aspect="auto"
    )
    ax2.set_title("Error Polygon Locations", fontweight="bold")
    ax2.set_xlabel("Longitude")
    ax2.set_ylabel("Latitude")

    # Add colorbar for error polygons
    cbar2 = plt.colorbar(im2, ax=ax2, shrink=0.8)
    cbar2.set_label("Recode Value")

    # 3. Corrected Data (simulated)
    ax3 = axes[1, 0]
    # Apply corrections for demonstration
    corrected_data = raster_data.copy()
    correction_mask = error_mask > 0

    # Apply different corrections based on recode values
    for recode_val in [1, 2, 3]:  # Different recode values
        mask = error_mask == (recode_val + 2)  # Offset applied earlier
        if np.any(mask):
            if recode_val == 1:  # Improve degraded areas
                corrected_data[mask] = np.clip(corrected_data[mask] + 0.5, -1, 1)
            elif recode_val == 2:  # Mark as stable
                corrected_data[mask] = 0
            elif recode_val == 3:  # Mark as degraded
                corrected_data[mask] = -0.7

    im3 = ax3.imshow(
        corrected_data,
        extent=extent,
        cmap=degradation_cmap,
        norm=degradation_norm,
        aspect="auto",
    )
    ax3.set_title("Corrected Land Degradation Status", fontweight="bold")
    ax3.set_xlabel("Longitude")
    ax3.set_ylabel("Latitude")

    # Add colorbar
    cbar3 = plt.colorbar(im3, ax=ax3, shrink=0.8)
    cbar3.set_label("Degradation Index")
    cbar3.set_ticks([-0.75, -0.3, 0, 0.3, 0.75])
    cbar3.set_ticklabels(["Degraded", "Declining", "Stable", "Improving", "Improved"])

    # 4. Difference Map (Before vs After)
    ax4 = axes[1, 1]
    difference = corrected_data - raster_data

    # Use diverging colormap for differences
    diff_cmap = plt.cm.RdBu_r
    max_diff = np.abs(difference).max()
    im4 = ax4.imshow(
        difference,
        extent=extent,
        cmap=diff_cmap,
        vmin=-max_diff,
        vmax=max_diff,
        aspect="auto",
    )
    ax4.set_title("Correction Impact (After - Before)", fontweight="bold")
    ax4.set_xlabel("Longitude")
    ax4.set_ylabel("Latitude")

    # Add colorbar for differences
    cbar4 = plt.colorbar(im4, ax=ax4, shrink=0.8)
    cbar4.set_label("Change in Degradation Index")

    # Add grid lines to all plots
    for ax in axes.flat:
        ax.grid(True, alpha=0.3)
        ax.tick_params(axis="both", which="major", labelsize=8)

    plt.tight_layout()
    plt.show()

    # Print summary statistics
    print("\\n📊 Correction Summary:")
    print(f"   Total pixels: {raster_data.size:,}")
    print(f"   Pixels corrected: {np.sum(correction_mask):,}")
    print(
        f"   Correction percentage: {100 * np.sum(correction_mask) / raster_data.size:.2f}%"
    )
    print(
        f"   Mean change in degradation index: {difference[correction_mask].mean():.3f}"
    )
    print(f"   Max absolute change: {np.abs(difference).max():.3f}")

else:
    print("❌ Cannot create maps - raster data not available")

## 9. Optional: Interactive Map with Folium

Create an interactive map to explore the results in detail (requires folium installation).

In [ ]:
# Create interactive map using folium (requires folium installation)
try:
    # Center the map on Antigua and Barbuda
    center_lat, center_lon = 17.1, -61.8

    # Create the map
    m = folium.Map(
        location=[center_lat, center_lon], zoom_start=12, tiles="OpenStreetMap"
    )

    # Add different base layers
    folium.TileLayer("Stamen Terrain").add_to(m)
    folium.TileLayer("CartoDB positron").add_to(m)

    # Add AOI boundary - fix coordinates access for Feature format
    aoi_coords = params["aoi"]["geometry"]["coordinates"][0]  # Fixed: added 'geometry'
    folium.Polygon(
        locations=[[coord[1], coord[0]] for coord in aoi_coords],
        color="blue",
        weight=2,
        fill=True,
        fillColor="blue",
        fillOpacity=0.1,
        popup="Area of Interest (AOI)",
    ).add_to(m)

    # Add error polygon markers
    error_features = params["error_polygons"]["features"]
    for i, feature in enumerate(error_features):
        coords = feature["geometry"]["coordinates"][0]
        properties = feature["properties"]
        location_name = properties.get("location_name", f"Error Polygon {i + 1}")
        process_change = properties.get("process_driving_change", "Unknown")
        basis = properties.get("basis_for_judgement", "No basis provided")
        recode_deg = properties.get("recode_deg_to", "None")
        recode_stable = properties.get("recode_stable_to", "None")
        recode_imp = properties.get("recode_imp_to", "None")

        # Get center of polygon for marker
        lons = [coord[0] for coord in coords]
        lats = [coord[1] for coord in coords]
        center_poly = [np.mean(lats), np.mean(lons)]

        # Create popup content
        popup_content = f"""
        <b>{location_name}</b><br>
        <b>Process:</b> {process_change}<br>
        <b>Basis:</b> {basis}<br>
        <b>Recode Settings:</b><br>
        &nbsp;&nbsp;Degraded → {recode_deg}<br>
        &nbsp;&nbsp;Stable → {recode_stable}<br>
        &nbsp;&nbsp;Improved → {recode_imp}
        """

        # Add polygon outline
        folium.Polygon(
            locations=[[coord[1], coord[0]] for coord in coords],
            color="red",
            weight=3,
            fill=True,
            fillColor="red",
            fillOpacity=0.3,
            popup=popup_content,
        ).add_to(m)

        # Add marker at center
        folium.Marker(
            center_poly,
            popup=popup_content,
            icon=folium.Icon(color="red", icon="exclamation-sign"),
        ).add_to(m)

    # Add layer control
    folium.LayerControl().add_to(m)

    # Add a legend
    legend_html = """
    <div style="position: fixed; 
                bottom: 50px; left: 50px; width: 200px; height: 90px; 
                background-color: white; border:2px solid grey; z-index:9999; 
                font-size:14px; padding: 10px
                ">
    <b>Legend</b><br>
    <i class="fa fa-square" style="color:blue"></i> Area of Interest<br>
    <i class="fa fa-square" style="color:red"></i> Error Polygons<br>
    <i class="fa fa-map-marker" style="color:red"></i> Error Locations
    </div>
    """
    m.get_root().html.add_child(folium.Element(legend_html))

    # Display the map
    print("🗺️ Interactive map created successfully!")
    print("   - Blue outline: Area of Interest")
    print("   - Red polygons: Error correction areas")
    print("   - Red markers: Error polygon centers")
    print("   - Click on polygons/markers for details")

    # Show the map
    m

except ImportError:
    print("⚠️ Folium not installed. Install with: pip install folium")
    print("Skipping interactive map creation...")
except Exception as e:
    print(f"❌ Error creating interactive map: {e}")
    print("Interactive map creation failed, but other visualizations should work.")

## 10. Conclusion and Next Steps

This notebook demonstrates the complete workflow for using the SDG 15.3.1 Error Recode algorithm:

### What We Accomplished:
✅ **API Authentication** - Successfully connected to trends.earth API  
✅ **Job Submission** - Submitted error recode job with polygon corrections  
✅ **Result Monitoring** - Tracked job progress and retrieved results  
✅ **Data Visualization** - Created comprehensive maps showing before/after analysis  
✅ **Interactive Exploration** - Generated interactive map for detailed investigation

### Key Features of the Enhanced Error Recode Script:
- **Multi-period Support**: Handles jobs with multiple time periods using filters parameter
- **Simplified Band Selection**: Uses existing get_band_by_name() filtering functionality
- **Error Polygon Integration**: Applies user-defined corrections to specific geographic areas
- **Robust Error Handling**: Clear error messages and graceful failure handling

### Next Steps:
1. **Customize Parameters**: Modify the error polygons and recode values for your specific use case
2. **Real Data Integration**: Replace synthetic data with actual S3 URIs from your analysis
3. **Advanced Analysis**: Combine with other SDG 15.3.1 indicators (productivity, land cover)
4. **Automation**: Integrate into larger workflows for operational monitoring

### Related Resources:
- **Statistics Analysis**: See `sdg_15_3_1_stats_example.ipynb` for complementary statistics calculation
- **API Documentation**: trends.earth API reference for additional endpoints
- **Algorithm Details**: SDG 15.3.1 methodology documentation

### Support:
For questions about this workflow or the trends.earth platform, consult the documentation or contact the development team.